In [20]:
import pandas as pd
import numpy as np
import math
from math import sin, cos, tan, atan, sqrt, atan2,pi
path = r'C:\a_sem_ii\SNS\temat_5_o1\o_1_kat.txt'

In [21]:
value = np.loadtxt(path)
value

array([ 2.40000000e+01,  1.50000000e+01,  6.00000000e+00,  6.00000000e+00,
        1.20000000e+01,  0.00000000e+00,  0.00000000e+00, -4.94960696e-05,
       -6.82121026e-13,  0.00000000e+00,  9.80000000e+01,  9.08437500e+01,
        4.58876270e-09,  2.03977848e+00,  4.74229455e-06,  3.24033631e-03,
        9.40449536e-06,  5.15367776e+03,  5.61600000e+05, -8.56816769e-08,
        1.06373984e+00,  8.94069672e-08,  9.53438415e-01,  1.95500000e+02,
        3.61536369e-01, -8.05712119e-09, -4.71448193e-11,  1.00000000e+00,
        1.84700000e+03,  0.00000000e+00,  2.00000000e+00,  0.00000000e+00,
        2.32830644e-09,  9.80000000e+01,  5.54340000e+05,  0.00000000e+00])

In [22]:
param = ['prn', 'rok','miesiac','dzien','godz','min','sek','a0','a1','a2','IODE','Crs','DELTAn','M0','Cuc','e','Cus','sqrt(A)','Toe','Cic','OMEGA0','Cis','I0','Crc','omega','OMEGADOT','Idot','L2','week','L2P','Accur','Health','Tgd','IODC','T_transm','fit']
rinex = dict(zip(param,value))

In [23]:
prn = rinex['prn']
rok = 2000+rinex['rok']
mies = rinex['miesiac']
dzien = rinex['dzien']

#czas rinex
godz_r = rinex['godz']
minuta_r = rinex['min']
sek_r = rinex['sek']


a0 = rinex['a0']
a1 = rinex['a1']
a2 = rinex['a2']

#tsv liczymy od niedzieli (z sob na ndz) do dnia tygodnia+godziny




Crs = rinex['Crs'] #metry
dn = rinex['DELTAn'] #semi-circles/sec
Cuc = rinex['Cuc'] #rad
e = rinex['e']#dimensionals
sqrtA = rinex['sqrt(A)'] #sqrt(metr)
toe = rinex['Toe'] #sec
Cic = rinex['Cic'] #rad
omega0 = rinex['OMEGA0'] #semi-circles
Cis = rinex['Cis'] # radians
i0 = rinex['I0'] # semi-circles
Cre = rinex['Crc'] #meters
omega = rinex['omega'] # semi - circles
omegadot = rinex['OMEGADOT'] #semi - circles/sec
m0 = rinex['M0']
Cus = rinex['Cus']
Crc = rinex['Crc']
Idot = rinex['Idot']

prn, rok, mies, dzien

(24.0, 2015.0, 6.0, 6.0)

In [48]:
u = 3.986005*10**14
omega_e = 7.2921151467*10**-5

#Czas na który liczymy
dzien_tyg_tsv = 6
godz_tsv = 11
min_tsv = 14
tsv = dzien_tyg_tsv*24*60*60+godz_tsv*60*60+min_tsv*60

#czas rinex
godz_r = rinex['godz']
minuta_r = rinex['min']
sek_r = rinex['sek']
toc = dzien_tyg_tsv*24*60*60+godz_r*60*60+minuta_r*60

dts = tsv-toc

dt = a0 + a1*(dts)+ a2*(dts)**2
t = tsv - dt
toc, tsv, dts

(561600.0, 558840, -2760.0)

In [49]:
A = sqrtA**2
n0 = math.sqrt(u/A**3)
tk = dts-dt
n = n0+dn
mk = m0+n*tk

In [50]:
Ek = 0
Ek_1 = 10
while abs(Ek-Ek_1) >10**-15:
    Ek_1 = Ek
    Ek = mk + e*sin(Ek)

licznik_vk = ((sqrt(1-e**2))*sin(Ek))/(1-e*cos(Ek))
mianownik_vk = (cos(Ek)-e)/(1-e*cos(Ek))
vk = atan2(licznik_vk,mianownik_vk)

fik = vk+omega
duk = Cus*sin(2*fik)+Cuc*cos(2*fik)
drk = Crs*sin(2*fik)+Crc*cos(2*fik)
dik = Cis*sin(2*fik)+Cic*cos(2*fik)

uk = fik+duk
rk = A*(1-e*cos(Ek))+drk
ik = i0+dik+Idot*tk

xkp = rk*cos(uk)
ykp = rk*sin(uk)

omk = omega0 + (omegadot-omega_e)*tk-omega_e*toc+4*2*pi
xk = xkp*cos(omk)-ykp*cos(ik)*sin(omk)
yk = xkp*sin(omk) + ykp*cos(ik)*cos(omk)
zk = ykp*sin(ik)
round(prn),round(toc), round(tsv,0), round(xk,3),round(yk,3),round(zk,3)

(24, 561600, 558840, 17286078.463, 4562510.659, 19650377.495)